## Análisis exploratorio de los datos

In [1]:
import pandas as pd

#df = pd.read_csv('/content/country_vaccinations.csv')
df = pd.read_csv('/content/sample_data/country_vaccinations.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36736 entries, 0 to 36735
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              36736 non-null  object 
 1   iso_code                             36736 non-null  object 
 2   date                                 36736 non-null  object 
 3   total_vaccinations                   20250 non-null  float64
 4   people_vaccinated                    19330 non-null  float64
 5   people_fully_vaccinated              16467 non-null  float64
 6   daily_vaccinations_raw               16671 non-null  float64
 7   daily_vaccinations                   36494 non-null  float64
 8   total_vaccinations_per_hundred       20250 non-null  float64
 9   people_vaccinated_per_hundred        19330 non-null  float64
 10  people_fully_vaccinated_per_hundred  16467 non-null  float64
 11  daily_vaccinations_per_milli

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


La base de datos contiene 33,725 registros y 15 columnas, dichas columnas tienen nombres que cumplen con las convenciones de Python. 

Algunas de nuestras columnas parecen no tener el tipo de dato correcto, por ejemplo la columna *date* es de tipo object en lugar de un *datetime*, sin embargo se hablará más a profundidad de ello en la sección de *Transformación de datos*.

Al observar los primeros registros de la base de datos es posible apreciar la existencia de NaN, lo que indica que la base de datos deberá limpiarse para poder realizar análisis apropiadamente. Esto se abordará en la siguiente sección.

## Limpieza de datos
Se eliminarán las filas que no contengan los datos más relevantes, es decir, aquellos en los que no se dispone de información oficial acerca de las siguientes variables:

*   total_vaccinations: Total de vacunas aplicadas a la fecha
*   people_vaccinated: Total de personas con al menos una dosia (independientemente de su esquema de vacunación, de acuerdo a la marca) a la fecha
*   people_fully_vaccinated: Total de personas cuyo esquema de vacunación ha sido completado a la fecha
<br><br>
Pasamos de 36,901 a 16,505 registros

In [2]:
df = df.dropna(subset=['total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated'])
df = df.reset_index(drop=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16400 entries, 0 to 16399
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              16400 non-null  object 
 1   iso_code                             16400 non-null  object 
 2   date                                 16400 non-null  object 
 3   total_vaccinations                   16400 non-null  float64
 4   people_vaccinated                    16400 non-null  float64
 5   people_fully_vaccinated              16400 non-null  float64
 6   daily_vaccinations_raw               13987 non-null  float64
 7   daily_vaccinations                   16385 non-null  float64
 8   total_vaccinations_per_hundred       16400 non-null  float64
 9   people_vaccinated_per_hundred        16400 non-null  float64
 10  people_fully_vaccinated_per_hundred  16400 non-null  float64
 11  daily_vaccinations_per_milli

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-05-11,504502.0,448878.0,55624.0,NaN,13921.0,1.30,1.15,0.14,358.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-05-20,547901.0,470341.0,77560.0,NaN,4822.0,1.41,1.21,0.20,124.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-05-24,573277.0,476367.0,96910.0,NaN,5692.0,1.47,1.22,0.25,146.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-05-26,590454.0,479372.0,111082.0,NaN,6768.0,1.52,1.23,0.29,174.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-05-27,593313.0,479574.0,113739.0,2859.0,6487.0,1.52,1.23,0.29,167.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


Como se puede apreciar, la columna *daily_vaccinations_raw* aún presenta varios NaN, esto es debido a que dicha columna indica el número de vacunas administradas diariamente (sin tratar de distribuir vacunas en periodos más largos), sin embargo por diversos factores muchos países no pueden ofrecer datos diariamente, como por ejemplo Afganistán al visualizar los primeros registros.

Vamos a verificar si existen más columnas con valores NaN.

In [3]:
df.isna().sum()

country                                   0
iso_code                                  0
date                                      0
total_vaccinations                        0
people_vaccinated                         0
people_fully_vaccinated                   0
daily_vaccinations_raw                 2413
daily_vaccinations                       15
total_vaccinations_per_hundred            0
people_vaccinated_per_hundred             0
people_fully_vaccinated_per_hundred       0
daily_vaccinations_per_million           15
vaccines                                  0
source_name                               0
source_website                            0
dtype: int64

Como se puede apreciar, existen 15 registros NaN en la columna daily_vaccinations (y en su equivalente per_million). Estos datos podrían ser rellenados con valor 0, debido a que representan un porcentaje muy pequeño del total de registros. Por otro lado, la columna daily_vaccinations_raw posee casi un 15% de valores NaN, por lo que se decidió eliminarla de la base de datos, ya que existe la columna daily_vaccinations que ofrece información similar, pero no presenta valores NaN.

In [4]:
df['daily_vaccinations'] = df['daily_vaccinations'].fillna(0)
df['daily_vaccinations_per_million'] = df['daily_vaccinations_per_million'].fillna(0)
df = df.drop(['daily_vaccinations_raw'], axis=1)
df.isna().sum()

country                                0
iso_code                               0
date                                   0
total_vaccinations                     0
people_vaccinated                      0
people_fully_vaccinated                0
daily_vaccinations                     0
total_vaccinations_per_hundred         0
people_vaccinated_per_hundred          0
people_fully_vaccinated_per_hundred    0
daily_vaccinations_per_million         0
vaccines                               0
source_name                            0
source_website                         0
dtype: int64

Finalmente nuestra base de datos ya está limpia. El siguiente paso será revisar si las diferentes variables poseen el tipo de variable apropiado. Vamos a darle un vistazo a los primeros registros de la base de datos y a su información.

In [5]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16400 entries, 0 to 16399
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              16400 non-null  object 
 1   iso_code                             16400 non-null  object 
 2   date                                 16400 non-null  object 
 3   total_vaccinations                   16400 non-null  float64
 4   people_vaccinated                    16400 non-null  float64
 5   people_fully_vaccinated              16400 non-null  float64
 6   daily_vaccinations                   16400 non-null  float64
 7   total_vaccinations_per_hundred       16400 non-null  float64
 8   people_vaccinated_per_hundred        16400 non-null  float64
 9   people_fully_vaccinated_per_hundred  16400 non-null  float64
 10  daily_vaccinations_per_million       16400 non-null  float64
 11  vaccines                    

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-05-11,504502.0,448878.0,55624.0,13921.0,1.30,1.15,0.14,358.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-05-20,547901.0,470341.0,77560.0,4822.0,1.41,1.21,0.20,124.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-05-24,573277.0,476367.0,96910.0,5692.0,1.47,1.22,0.25,146.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-05-26,590454.0,479372.0,111082.0,6768.0,1.52,1.23,0.29,174.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-05-27,593313.0,479574.0,113739.0,6487.0,1.52,1.23,0.29,167.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


## Transformación de datos
Según la documentación de 'Pandas', algunas de las columnas de nuestro dataset son candidatas para ser de tipo 'category', ya que contienen un número limitado de valores posibles, como en el caso de los paises y de las organizaciones que proporcionan los datos contenidos en nuestro dataset.
<br><br>
Dicha documentación nos explica que algunas de las ventajas de usar este tipo de dato es que ahorra memoria y que, sobre todo, las columnas convertidas permitirán que ciertas bibliotecas las interpreten más facilmente como variables categóricas al momento de realizar gráficas o utilizar métodos estadísticos.


In [6]:
tipos_nuevos = {
    'country': 'category',
    'iso_code': 'category',
    'source_name': 'category'
}

df = df.astype(tipos_nuevos)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16400 entries, 0 to 16399
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              16400 non-null  category      
 1   iso_code                             16400 non-null  category      
 2   date                                 16400 non-null  datetime64[ns]
 3   total_vaccinations                   16400 non-null  float64       
 4   people_vaccinated                    16400 non-null  float64       
 5   people_fully_vaccinated              16400 non-null  float64       
 6   daily_vaccinations                   16400 non-null  float64       
 7   total_vaccinations_per_hundred       16400 non-null  float64       
 8   people_vaccinated_per_hundred        16400 non-null  float64       
 9   people_fully_vaccinated_per_hundred  16400 non-null  float64       
 10  daily_vacc

Tras todo el proceso de limpieza de datos, el tamaño de la base de datos paso de 4.2 MB a solo 1.8 MB, y con el proceso de transformación de datos, se redujo de 1.8 MB a solo 1.5 MB. 

Es decir, el proceso de limpieza logro disminuir el tamaño de la base en más de 50% y ofrece la futura facilidad para analizar los datos, mientras que el proceso de transformar los datos en las variables adecuadas redujo en un 16% el tamaño de la base ya limpia. Esto es uno de los beneficios de realizar ambos procesos.

## Resolución de preguntas

Desde los primeros pasos del proyecto se plantearon algunas preguntas que pudieran ser de interés para el tema de investigación. Ahora que se dispone de una base de datos limpia, ha llegado la hora de responderlas.

### ¿En qué país el programa de vacunación está más avanzado?

In [7]:
df[df['total_vaccinations_per_hundred'] == df['total_vaccinations_per_hundred'].max()]['country']

5529    Gibraltar
Name: country, dtype: category
Categories (214, object): ['Afghanistan', 'Albania', 'Algeria', 'Andorra', ..., 'Wallis and Futuna',
                           'Yemen', 'Zambia', 'Zimbabwe']

Gibraltar propiamente no es un país, sino un territorio británico de Ultramar con una población estimada de 33,701 personas en 2019. Esto explicaría por que es el territorio con la campaña de vacunación más rápida, debido a su escasa población y a pertenecer al Reino Unido, uno de los países que se ha caracterizado por una efectiva campaña de vacunación.

### ¿Dónde hay más personas vacunadas por día? 

In [8]:
df[df['daily_vaccinations_per_million'] == df['daily_vaccinations_per_million'].max()]['country']

1529    Bhutan
Name: country, dtype: category
Categories (214, object): ['Afghanistan', 'Albania', 'Algeria', 'Andorra', ..., 'Wallis and Futuna',
                           'Yemen', 'Zambia', 'Zimbabwe']

Bután es un país de Asia del Sur con una población de 763,092 al año 2019. El resultado obtenido nos indica que Bután consiguió vacunar a 0.15% de su población en un solo día en la parte más avanzada de la campaña de vacunación.

### ¿Cuál es el rating de vacunas más difundidas a nivel mundial? 

In [9]:
df.groupby('vaccines').size().sort_values(ascending=False).head(5)

vaccines
Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech        3448
Moderna, Oxford/AstraZeneca, Pfizer/BioNTech                         2037
Moderna, Pfizer/BioNTech                                             1341
Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V     894
Oxford/AstraZeneca, Pfizer/BioNTech                                   745
dtype: int64

Este indicador nos muestra que conjunto de vacunas es aplicado mayormente por los países, siendo la combinación de Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech la que más predomina. Sin embargo, es importante notar que hay un registro de estas combinaciones de vacunas por cada día que determinado país reporta, sin embargo, nos sigue dando una idea apropiada debido a que los países que reportan con mayor frecuencia son aquellos que tienen una mejor organización en su campaña de vacunación.

### ¿Qué países reportan sus datos con mayor frecuencia?

In [10]:
df.groupby('country').size().sort_values(ascending=False).head(5)


country
Scotland       239
Germany        226
Switzerland    224
Belgium        223
England        221
dtype: int64

Como podemos apreciar, los 5 países que con mayor frecuancia reportan información sobre el avanza de la campaña de vacunación son todos europeos: Escocia (país constituyente del Reino Unido), Alemania, Suiza, Bélgica y Reino Unido. En este indicador influye tanto la periodicidad como la fecha de inicio de vacunación.

### ¿Cuáles son los sitios web que brindan mayor información? 

In [11]:
df.groupby('source_website').size().sort_values(ascending=False).head(5)

source_website
https://coronavirus.data.gov.uk/details/vaccinations                  1080
https://covid19.who.int/                                               545
https://impfdashboard.de/                                              226
https://www.covid19.admin.ch/en/epidemiologic/vacc-doses?detGeo=CH     224
https://epistat.wiv-isp.be/covid/                                      223
dtype: int64

Finalmente, este indicador nos revela de que sitios se consulta más veces la información. Con la excepción de los primeros dos sitios webs, existe una relación entre los últimos dos indicadores, ya que los tercer. cuarto y quinto sitios web más consultados son los que corresponden a Alemania, Suiza y Bélgica. El segundo sitio web más consultado es el de la Organización Mundial de la Salud, lo cual tiene sentido pues también dispone de información de muchos países en caso de no disponer con cifras de una nación determinada, mientras que el primer lugar por mucha diferencia es el Reino Unido, debido a que cada país constituyente (Inglaterra, Gales, Escocia e Irlanda del Norte) reportan por separado.

## Conclusión

Analizando los indicadores calculados previamente, es posible observa la situación en algunos de los países con mejores avances en sus campañas de vacunación. Es importante analizar la situación epidemiológica actual de estos países debido a la aparición de nuevas variables que podían reducir la eficacia de las vacunas al prevenir principalmente hospitalizaciones y muertes que pudieran colapsar los sistemas de salud alrededor del mundo, y esto podría provocar que el estado de pandemia se extienda aún más, con todas las consecuencias negativas tanto a nivel salud como a nivel económico que eso implicaría.